In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import csv
import time
import random
import re
from collections import defaultdict

#options = webdriver.ChromeOptions()
#options.add_argument('--proxy-server=112.140.186.170:808')
#driver = webdriver.Chrome(executable_path='C:/Users/KNguyen/chromedriver.exe', chrome_options=options)

In [5]:
def hasNumbers(inputString):
    return bool(re.search(r'\d+', inputString))

columns = defaultdict(list)

with open('C:/Users/KNguyen/csv_outputs/NCUC_master_list.csv') as ncuc:
    reader = csv.DictReader(ncuc)
    for row in reader:
        for (k,v) in row.items(): 
            columns[k].append(v)
            
docket_details = columns['Detail']
format_details = [x for x in docket_details if x!='none']

In [6]:
driver = webdriver.Chrome("C:/Users/KNguyen/chromedriver.exe")
with open('C:/Users/KNguyen/csv_outputs/NCUC_ServiceList_test.csv', 'w', newline='') as ncuc:
    a = csv.writer(ncuc, delimiter=',')
    bill_rows = [['ID', 'Party', 'Address', 'Fax', 'Telephone', 'E-Mail', 'Date Added', 'Recorded Date']]
    
    for docket in format_details:
        try:
            driver.get(docket)
            driver.find_element_by_link_text('Service List').click()

            driver.implicitly_wait(2)
            results_dirty = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_Content > table > tbody > tr:nth-child(18) > td > div > div > table > tbody")
            results_html = results_dirty.get_attribute("innerHTML")
            service_items_dirty = results_html.split('<tbody>')[1:]
            
            docket_id_dirty = driver.find_element_by_class_name("pageTitle").text
            docket_id = docket_id_dirty.replace('Docket ','')
            
            pattern=re.compile('\n|\t|<.*>|  ')

            for item in service_items_dirty:
                test2 = item.replace('<td>','</td>').replace('<td','</td>').split('</td>')
                test1=[]
                for item in test2:
                    test3 = pattern.sub('', item)
                    test1.append(test3) 
                test1 = list(filter(None, test1))  
                #print (test2)
                
                email_address = ''
                for item in test2:
                    match = re.search('([\w.-]+)@([\w.-]+)', item)
                    if match:
                        email_address = match.group()
                        #print (email_address)
                
                party_with_address = []
                other_parts=re.compile('.*Fax.*|.*Telephone.*|.*>.*|&nbsp')
                name_w_numbers=''

                for i in range (0,len(test1)):
                    if hasNumbers (test1[i]) is False:
                        sample1=test1[i]
                        party_with_address.append(sample1)

                party_dirty=[]        
                for item in party_with_address:
                    test4 = other_parts.sub('', item)
                    party_dirty.append(test4) 

                address_dirty = []
                for i in range (0,len(test1)):
                    if hasNumbers (test1[i]) is True:
                        sample1=test1[i]
                        address_dirty.append(sample1)
                address_list=[]        
                for item in address_dirty:
                    test5 = other_parts.sub('', item)
                    address_list.append(test5) 

                for item in address_list:
                    match = re.search('.*LLC.*|.*INC.*|.*SOLAR.*', item)
                    if match:
                        name_w_numbers = match.group()

                if name_w_numbers not in party_dirty:
                    party_dirty.insert(0,name_w_numbers)  
                party_clean = ''
                for i in range (0,len(party_dirty)):
                    party_clean += party_dirty[i] + ' '

                if name_w_numbers in address_list:
                    address_list.remove(name_w_numbers)
                address_clean = ''
                for i in range (0,len(address_list)):
                    address_clean += address_list[i] + ' '

                fax_num = ''
                tel_num = ''           
                for item in test1:
                    if ('Fax' in item):
                        if ('Telephone' in item):
                            test6 = item.replace('Telephone',';').split(';')
                            fax_num = test6[2]
                            tel_num = test6[-1]
                            #print ("Fax: " + fax_num)
                            #print ("Tel: " + tel_num)
                    elif ('Telephone' in item):
                        test7 = item.split(';')
                        tel_num = test7[-1]
                        #print ("Tel: " + tel_num)

                date_added = ''
                for item in test1:
                    match = re.search('(\d+/\d+/\d+)', item)
                    if match:
                        date_added = match.group()
                        #print (date_added)

                party_of_record = ''
                for item in test1:
                    if ('No' in item):
                        party_of_record = 'No'
                        #print (party_of_record)
                    if ('Yes' in item):
                        party_of_record = 'Yes'
                        #print (party_of_record)
                #print()
                newrow = [docket_id, party_clean, address_clean, fax_num, tel_num,email_address,date_added,party_of_record]
                bill_rows.append(newrow)
        except NoSuchElementException:
            driver.refresh()
            continue
    a.writerows(bill_rows)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=53.0.2785.116)
  (Driver info: chromedriver=2.9.248315,platform=Windows NT 6.3 x86_64)
